In [2]:
import sys
import math

# input files
bsmFile = open("0bsmqtest.txt")     # file containing BSM and sender/receiver data
sentFile = open("0sentqtest.txt")   # file containing number of sent and received BSMs
idFile = open("0idsqtest.txt")      # file containing values for mobility->getExternalId() and getParentModule()->getIndex()
cbrFile = open("0cbrqtest.txt")     # file containing CBR data
ipdFile = open("0ipdqtest.txt")     # file containing distance and position stats
densityFile = open("0densityqtest.txt")

# output file
writeToFile = True
outpFileName = "output.txt" # output file name

if writeToFile:
    outpFile = open(outpFileName, "w")

# PART 1: Calculate Sent/Received/Lost BSMs

bsmDict = {}
totalSent = 0
totalReceived = 0
totalUnique = 0

sentLine = sentFile.readline().strip('\n')
while sentLine:
        first = sentLine.find('\t')
        second = sentLine.find('\t', first+1)
        vId = sentLine[:first]
        numSent = sentLine[first+1:second]
        numReceived = sentLine[second+1:]
        totalSent += eval(numSent)
        totalReceived += eval(numReceived)
        
        sublist = [i for i in range(1, eval(numSent)+1)]
        bsmDict[vId] = sublist
        sentLine = sentFile.readline().strip('\n')

vDict = {} # this will contain the link between x and y in each line in the ID file
numVehs = 0

idLine = idFile.readline().strip('\n')
while idLine:
        u = idLine.find('_')
        vId = idLine[u+1:] # this is the node[] value, from getParentModule()->getIndex()
        fId = idLine[0:u] # this is the mobility->getExternalId() value
        vDict[fId] = vId
        numVehs += 1
        idLine = idFile.readline().strip('\n')

j = 0
bsmLine = bsmFile.readline().strip('\n')
print("\nCalculating Lost BSMs...")
while bsmLine:
        first = bsmLine.find('_')
        second = bsmLine.find('_', first+1)
        third = bsmLine.find('_', second+1)
        receiverID = vDict[bsmLine[:first]]
        senderID = vDict[bsmLine[first+1:second]]
        bsmID = eval(bsmLine[second+1:third])
        if bsmID in bsmDict[senderID]:
                bsmDict[senderID].remove(bsmID)
                totalUnique += 1
        bsmLine = bsmFile.readline().strip('\n')
        j+=1
        if j%10000 == 0:
                print('\r' + str(j), end="")
print('\r' + str(j) + "... Done!")
        
#print(bsmList)
numLost = 0
for v in bsmDict.keys():
        #print(len(bsmDict[v]))
        numLost += len(bsmDict[v])

if writeToFile:
    outpFile.write("Total Sent: " + str(totalSent))
    outpFile.write("\nTotal Received (not unique): " + str(totalReceived))
    outpFile.write("\nTotal Received (unique): " + str(totalUnique))
    outpFile.write("\nTotal Lost: " + str(numLost))
    outpFile.write("\nBER (lost/sent): " + str(numLost/totalSent))
else:
    print("Total Sent:", totalSent)
    print("Total Received (not unique):", totalReceived)
    print("Total Received (unique):", totalUnique)
    print("Total Lost:", numLost)
    print("BER (lost/sent):", numLost/totalSent)

bsmFile.close()
sentFile.close()
idFile.close()

########################################################

# PART 2: Calculate Average CBR

cbrLine = cbrFile.readline().strip('\n')

# restrictions for start/end time
#startTime = -1
#endTime = 999999

sumCBR = 0
numVals = 0
j = 0
print("\nCalculating Average CBR...")
while cbrLine:
        first = cbrLine.find('\t')
        second = cbrLine.find('\t', first+1)
        third = cbrLine.find('\t', second+1)
        
        simTime = eval(cbrLine[first+1:second])
        cbr = eval(cbrLine[third+1:])
        
        #if startTime <= simTime <= endTime:
        sumCBR += cbr
        numVals += 1
        
        j+=1
        if j%10000 == 0:
                print('\r' + str(j), end="")
        cbrLine = cbrFile.readline().strip('\n')
print('\r' + str(j) + "... Done!")

if writeToFile:
    outpFile.write("\nAverage CBR: " + str(sumCBR/numVals))
else:
    print("Average CBR:", sumCBR/numVals)

cbrFile.close()

#########################################################

# PART 3: Calculate Average Density

densityLine = densityFile.readline().strip('\n')
densitySum = 0
numVals = 0
j = 0
print("\nCalculating Average Density...")
while densityLine:
    content = densityLine.split()
    simTime = eval(content[1])
    vehID = content[2]
    density = eval(content[3])
    if simTime >= 1.1:
        densitySum += density
        numVals += 1        
    j+=1
    if j%10000 == 0:
        print('\r' + str(j), end="")
    densityLine = densityFile.readline().strip('\n')
print('\r' + str(j) + "... Done!")

if writeToFile:
    outpFile.write("\nAverage Density:\t" + str(densitySum/numVals))
else:
    print("Average Density:\t" + str(densitySum/numVals))

densityFile.close()

#########################################################

# PART 4: Calculate IPD


ipdLine = ipdFile.readline().strip('\n')
        
ipdList = [] # contains IPD data for each sender-receiver vehicle pair
for i in range(0, numVehs):
        ipdList.append([-1 for j in range(0, numVehs)])
#print("Now scanning input")

# this file can be used to write each IPD and distance value to; can be commented out if not needed
#output = open("output4lanetenhz20mw200.txt", "w")

# distance ranges; these are optional and can be commented out
# to be used when you want the total number of BSMs received within 50m,
# 100m, 200m, 300m, and >300m by all vehicles
n0 = 0
n50 = 0
n100 = 0
n200 = 0
n300 = 0

# this list is to be used if you want to get the average IPD for consistent
# distance intervals; number of intervals can be changed as needed.
ipdIntervals = [[] for i in range(0,200)]

# individual vehicle IPD data; these are optional and can be commented out
# to be used when you want to get IPD data on specific vehicles
ipd50 = []
ipd100 = []
ipd150 = []
ipd200 = []
ipd539 = []

maxDist = 0
maxGap = 0
last = -1

j = 1
print("\nCalculating IPD...")
while ipdLine:
        if ipdLine[:10] == 'undisposed':
                ipdFile.readline().strip('\n')
                continue
        
        # divides each line in the distance file into the individual data pieces
        first = ipdLine.find('_')
        second = ipdLine.find('_', first+1)
        third = ipdLine.find('_', second+1)
        fourth = ipdLine.find('_', third+1)
        fifth = ipdLine.find('_', fourth+1)
        sixth = ipdLine.find('_', fifth+1)
        seventh = ipdLine.find('_', sixth+1)
        senderX = eval(ipdLine[:first])
        senderY = eval(ipdLine[first+1:second])
        senderID = ipdLine[second+1:third]
        receiverX = eval(ipdLine[third+1:fourth])
        receiverY = eval(ipdLine[fourth+1:fifth])
        receiverID = ipdLine[fifth+1:sixth]
        simTime = eval(ipdLine[seventh+1:])
        distance = round(math.sqrt((senderX-receiverX)**2 + (senderY-receiverY)**2), 5) # calculates distance between sending and receiving vehicle
        
        """if last == -1:
                last = simTime
        else:
                if simTime - last > maxGap:
                        maxGap = simTime - last
                last = simTime"""

# this block is only to be used with the distance range variables; otherwise it can be commented out
# the distance values can be changed as needed
        if distance <= 50:
                n0 += 1
        elif distance <= 100:
                n50 += 1
        elif distance <= 200:
                n100 += 1
        elif distance <= 300:
                n200 += 1
        else:
                n300 += 1
        
        lastTime = ipdList[eval(vDict[receiverID])][eval(vDict[senderID])]
        if lastTime != -1:
                ipd = simTime - lastTime
        else:
                ipd = -1
                
        if distance > maxDist:
                maxDist = distance
                
# this section is only to be used with the individual vehicle IPD data; otherwise it can be commented out
# vehicle names and if condition can be changed as needed
                """if senderID == 'sender' and 'receiver' in receiverID:
                        if receiverID == 'receiver50':
                                ipd50.append(ipd)
                        elif receiverID == 'receiver100':
                                ipd100.append(ipd)
                        elif receiverID == 'receiver150':
                                ipd150.append(ipd)
                        elif receiverID == 'receiver200':
                                ipd200.append(ipd)
                        elif receiverID == 'receiver539':
                                ipd539.append(ipd)"""
                        
# this section is only used if you are writing the IPD and distance values
# to the output file; otherwise it can be commented out
                """output.write(senderID + '\t' + receiverID + '\t' + str(distance) + '\t' + str(ipd) + '\n')
                output.flush()"""
                
# this section is used only if you want to get the average IPD over consistent
# distance intervals; otherwise it can be commented out
# the number of intervals and length of each interval can be changed as needed
        for i in range(1, 201):
                if 20*i > distance and ipd != -1:
                        ipdIntervals[i-1].append(ipd)
                        break

# this section is only used if you are writing the IPD and distance values
# to the output file and only if you want to include the first time a
# sender-receiver pair exchanges BSMs; otherwise it can be commented out		
        """else:
                output.write(senderID + '\t' + receiverID + '\t' + str(distance) + '\t' + "First" + '\n')"""
        
        
        ipdList[eval(vDict[receiverID])][eval(vDict[senderID])] = simTime
        ipdLine = ipdFile.readline().strip('\n')
        
        # this block tells you how many lines the program has gone through so you can
        # see how much progress the program is making
        j+=1
        if j%10000 == 0:
                print('\r' + str(j), end="")
print('\r' + str(j) + "... Done!")

# print IPD data for individual vehicles if used
"""print(sum(ipd50)/len(ipd50), len(ipd50))
print(sum(ipd100)/len(ipd100), len(ipd100))
print(sum(ipd150)/len(ipd150), len(ipd150))
print(sum(ipd200)/len(ipd200), len(ipd200))
print(sum(ipd539)/len(ipd539), len(ipd539))"""

# print number of BSMs received in distance ranges if used
#print(n0, n50, n100, n200, n300)

# print average IPD for each distance interval if ipdIntervals is used
#print("**********")
for i in range(0, 100):
        if len(ipdIntervals[i]) > 0:
                ipdIntervals[i] = sum(ipdIntervals[i])/len(ipdIntervals[i])
        else:
                ipdIntervals[i] = 'N/A'

if writeToFile:
    outpFile.write('\n\nIPD Intervals:\n')
    for i in ipdIntervals:
        outpFile.write(str(i) + '\n')
else:
    print('\n\nIPD Intervals:\n')
    for i in ipdIntervals:
        print(str(i) + '\n')

#print(maxGap)
#print(maxDist)

ipdFile.close()

outpFile.close()



Calculating Lost BSMs...
969724... Done!

Calculating Average CBR...
61872... Done!

Calculating Average Density...
245079... Done!

Calculating IPD...
969725... Done!
